The following establishes a connection to Mechanical Turk.   
Set the create_hits_in_production variable equal to true to public real tasks 

In [68]:
import boto3 
import xmltodict 
import json 

create_hits_in_production = False
environments = {
  "production": {
    "endpoint": "https://mturk-requester.us-east-1.amazonaws.com",
    "preview": "https://www.mturk.com/mturk/preview"
  },
  "sandbox": {
    "endpoint": 
          "https://mturk-requester-sandbox.us-east-1.amazonaws.com",
    "preview": "https://workersandbox.mturk.com/mturk/preview"
  },
}
mturk_environment = environments["production"] if create_hits_in_production else environments["sandbox"]
session = boto3.Session(profile_name='mturk')
client = session.client(
    service_name='mturk',
    region_name='us-east-1',
    endpoint_url=mturk_environment['endpoint'],
)

print(client.get_account_balance()['AvailableBalance'])

10000.00


The below cell implements and creates HITs for the non-expert task. 

In [69]:
images = {"https://ids.si.edu/ids/deliveryService?max_w=800&id=SAAM-1983.95.99_2":"Homeward",
         "https://ids.si.edu/ids/deliveryService?max_w=800&id=FS-F1898.110_002":"Boy Viewing Mount Fuji",
         "https://ids.si.edu/ids/deliveryService?max_w=800&id=SAAM-1969.129_1":"Shimmering Tree Shadows",
         "https://ids.si.edu/ids/deliveryService?max_w=800&id=SAAM-1967.6.19_1":"The Thundershower",
         "https://ids.si.edu/ids/deliveryService?max_w=800&id=SAAM-1988.53_1":"Yliaster"}

html_layout = open('./arteval.html', 'r').read()
QUESTION_XML = """<HTMLQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2011-11-11/HTMLQuestion.xsd">
        <HTMLContent><![CDATA[{}]]></HTMLContent>
        <FrameHeight>650</FrameHeight>
        </HTMLQuestion>"""
question_xml = QUESTION_XML.format(html_layout)

TaskAttributes = {
    'MaxAssignments': 5,           
    # How long the task will be available on MTurk (1 hour)     
    'LifetimeInSeconds': 60*60,
    # How long Workers have to complete each item (10 minutes)
    'AssignmentDurationInSeconds': 60*10,
    # The reward you will offer Workers for each response
    'Reward': '0.05',                     
    'Title': 'Test 7',
    'Keywords': 'artwork, rating',
    'Description': 'Rate the artwork on a scale of 1 to 10.'
}

results = []
hit_type_id = ''
for image in images:
    response = client.create_hit(
        **TaskAttributes,
        Question=question_xml.replace('${content}',image)
    )
    hit_type_id = response['HIT']['HITTypeId']
    results.append({
        'image': images[image],
        'hit_id': response['HIT']['HITId']
    })
    
print("You can view the HITs here:")
print(mturk_environment['preview']+"?groupId={}".format(hit_type_id))

You can view the HITs here:
https://workersandbox.mturk.com/mturk/preview?groupId=3N79SUTLMAAKZH24FD8BK5LGHQJB5M


In [ ]:
for item in results:
    
    # Get the status of the HIT
    hit = client.get_hit(HITId=item['hit_id'])
    item['status'] = hit['HIT']['HITStatus']
    # Get a list of the Assignments that have been submitted
    assignmentsList = client.list_assignments_for_hit(
        HITId=item['hit_id'],
        AssignmentStatuses=['Submitted', 'Approved'],
        MaxResults=10
    )
    assignments = assignmentsList['Assignments']
    item['assignments_submitted_count'] = len(assignments)
    answers = []
    responses = [] 
    for assignment in assignments:
        # Retreive the attributes for each Assignment
        worker_id = assignment['WorkerId']
        assignment_id = assignment['AssignmentId']
        # Retrieve the value submitted by the Worker from the XML
        answer_dict = xmltodict.parse(assignment['Answer'])
        # print(answer_dict['QuestionFormAnswers']['Answer'][0]['FreeText']) # text 
        # print(answer_dict['QuestionFormAnswers']['Answer'][1]['FreeText']) # rating 
        response = answer_dict['QuestionFormAnswers']['Answer'][0]['FreeText']
        responses.append(response)
        answer = answer_dict['QuestionFormAnswers']['Answer'][1]['FreeText']
        answers.append(int(answer))
        
        # Approve the Assignment (if it hasn't been already)
        if assignment['AssignmentStatus'] == 'Submitted':
            client.approve_assignment(
                AssignmentId=assignment_id,
                OverrideRejection=False
            )
    
    # Add the answers that have been retrieved for this item
    item['answers'] = answers
    item['responses'] = responses 
    if len(answers) > 0:
        item['avg_answer'] = sum(answers)/len(answers)
print(json.dumps(results,indent=2))

The below cell implements and creates HITs for the expert task. 

In [73]:
images = {"https://ids.si.edu/ids/deliveryService?max_w=800&id=SAAM-1983.95.99_2":"Homeward",
         "https://ids.si.edu/ids/deliveryService?max_w=800&id=FS-F1898.110_002":"Boy Viewing Mount Fuji",
         "https://ids.si.edu/ids/deliveryService?max_w=800&id=SAAM-1969.129_1":"Shimmering Tree Shadows",
         "https://ids.si.edu/ids/deliveryService?max_w=800&id=SAAM-1967.6.19_1":"The Thundershower",
         "https://ids.si.edu/ids/deliveryService?max_w=800&id=SAAM-1988.53_1":"Yliaster"}

html_layout = open('./arteval2.html', 'r').read()
QUESTION_XML = """<HTMLQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2011-11-11/HTMLQuestion.xsd">
        <HTMLContent><![CDATA[{}]]></HTMLContent>
        <FrameHeight>650</FrameHeight>
        </HTMLQuestion>"""
question_xml = QUESTION_XML.format(html_layout)

TaskAttributes = {
    'MaxAssignments': 5,           
    # How long the task will be available on MTurk (1 hour)     
    'LifetimeInSeconds': 60*60,
    # How long Workers have to complete each item (10 minutes)
    'AssignmentDurationInSeconds': 60*10,
    # The reward you will offer Workers for each response
    'Reward': '0.05',                     
    'Title': 'Test 7',
    'Keywords': 'artwork, rating',
    'Description': 'Rate the artwork on a scale of 1 to 10.'
}

# TODO, add descriptions for each picture and replace the text in the html doc with the descriptions 

results = []
hit_type_id = ''
for image in images:
    response = client.create_hit(
        **TaskAttributes,
        Question=question_xml.replace('${content}',image)
    )
    hit_type_id = response['HIT']['HITTypeId']
    results.append({
        'image': images[image],
        'hit_id': response['HIT']['HITId']
    })
    
print("You can view the HITs here:")
print(mturk_environment['preview']+"?groupId={}".format(hit_type_id))

You can view the HITs here:
https://workersandbox.mturk.com/mturk/preview?groupId=3N79SUTLMAAKZH24FD8BK5LGHQJB5M


In [72]:
for item in results:
    
    # Get the status of the HIT
    hit = client.get_hit(HITId=item['hit_id'])
    item['status'] = hit['HIT']['HITStatus']
    # Get a list of the Assignments that have been submitted
    assignmentsList = client.list_assignments_for_hit(
        HITId=item['hit_id'],
        AssignmentStatuses=['Submitted', 'Approved'],
        MaxResults=10
    )
    assignments = assignmentsList['Assignments']
    item['assignments_submitted_count'] = len(assignments)
    answers = []
    responses = [] 
    for assignment in assignments:
        # Retreive the attributes for each Assignment
        worker_id = assignment['WorkerId']
        assignment_id = assignment['AssignmentId']
        # Retrieve the value submitted by the Worker from the XML
        answer_dict = xmltodict.parse(assignment['Answer'])
        # print(answer_dict['QuestionFormAnswers']['Answer'][0]['FreeText']) # text 
        # print(answer_dict['QuestionFormAnswers']['Answer'][1]['FreeText']) # rating 
        response = answer_dict['QuestionFormAnswers']['Answer'][0]['FreeText']
        responses.append(response)
        answer = answer_dict['QuestionFormAnswers']['Answer'][1]['FreeText']
        answers.append(int(answer))
        
        # Approve the Assignment (if it hasn't been already)
        if assignment['AssignmentStatus'] == 'Submitted':
            client.approve_assignment(
                AssignmentId=assignment_id,
                OverrideRejection=False
            )
    
    # Add the answers that have been retrieved for this item
    item['answers'] = answers
    item['responses'] = responses 
    if len(answers) > 0:
        item['avg_answer'] = sum(answers)/len(answers)
print(json.dumps(results,indent=2))

[
  {
    "image": "Homeward",
    "hit_id": "39XCQ6V3L5UV3ECRPUMNO96YNOD65K",
    "status": "Assignable",
    "assignments_submitted_count": 1,
    "answers": [
      5
    ],
    "avg_answer": 5.0,
    "responses": [
      "Select this if you feel extremely negative about the work.Select this if you feel extremely negative about the work.Select this if you feel extremely"
    ]
  },
  {
    "image": "Boy Viewing Mount Fuji",
    "hit_id": "34D9ZRXCZYKXWIM5R3G7XPICXRBASG",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "answers": [],
    "responses": []
  },
  {
    "image": "Shimmering Tree Shadows",
    "hit_id": "3W1K7D6QTI7ZG4DBS9ABOTZL2OVZB4",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "answers": [],
    "responses": []
  },
  {
    "image": "The Thundershower",
    "hit_id": "32CXT5U15NTZS7LUJARTGZU5S0W8UQ",
    "status": "Assignable",
    "assignments_submitted_count": 0,
    "answers": [],
    "responses": []
  },
  {
    "imag

In [ ]:
OrderedDict([('QuestionFormAnswers', OrderedDict([('@xmlns', 'http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd'), ('Answer', [OrderedDict([('QuestionIdentifier', 'description'), ('FreeText', 'asdfasdfasdfsad asdfasdasdf asdfasdfasdfasdfasdfsadfa')]), OrderedDict([('QuestionIdentifier', 'sentiment'), ('FreeText', '4')])])]))])